In [7]:
import pandas as pd
import requests

In [8]:
api_key = "f3898958-b76a-4a47-816f-0294f0c5103d"

BASE_URL = "https://api.hellodata.ai"

HEADERS = {
    "x-api-key": api_key
}

In [9]:
dimasset = pd.read_csv('data/DimAsset.csv')
dimasset = dimasset[dimasset['YearBuiltEnd'] < 2025]
assetdetailactive = pd.read_csv('data/vw_AssetDetailActive.csv', usecols=['AssetCode', 'ParentAssetName'])

dimasset = dimasset.merge(assetdetailactive, on='AssetCode')

In [10]:
display(dimasset.columns)

Index(['AssetName', 'AssetCode', 'InvestmentStrategy', 'AcquisitionType',
       'AcquisitionDate', 'SaleDate', 'IsTaxCredit', 'ActualStabilizationDate',
       'AreaType', 'AssetClass', 'AssetStyle', 'StabilizationStatus',
       'OccupancyStabilizationDate', 'CortosMSAID', 'CortosSubMarketID',
       'CoStarSubMarketID', 'CBSAID', 'oslPropertyID', 'oslSubPropertyID',
       'AverageBuildYear', 'YearBuiltEnd', 'YearRenovated', 'CurrentUnitCount',
       'RentableSquareFoot', 'ConstructionBeginDate', 'ConstructionEndDate',
       'Acreage', 'AcquisitionPrice', 'TotalHardCosts',
       'TotalSoftAndClosingCosts', 'TotalProjectCosts', 'IsSeniorHousing',
       'IsUnitReno', 'IsRecapitalization', 'ZipCode', 'Latitude', 'Longitude',
       'WebAddress', 'EmailAddress', 'osaEntityID', 'Phone', 'Address1',
       'PlannedUnitCount', 'LegalName', 'TaxID', 'Portfolio', 'City',
       'StateCode', 'ParentAssetName'],
      dtype='object')

In [11]:
def fetch_property_data(property, lat=None, lon=None, zip_code=None):
    """Function to fetch property data using property name and zip code."""
    querystring = {"q": property}

    # Only add lat and lon if they are provided
    if lat is not None and lon is not None:
        querystring["lat"] = lat
        querystring["lon"] = lon
        querystring["max_distance"] = 0.2

    if zip_code is not None:
        querystring['zip_code'] = zip_code

    url = f"{BASE_URL}/property/search"
    response = requests.get(url, headers=HEADERS, params=querystring)
    try:
        data = response.json()
        return data if data and len(data) > 0 else None
    except ValueError as e:
        raise ValueError(f"Error parsing JSON response from property search: {e}")

def fetch_property_details(property_id):
    """Function to fetch details for a specific property."""
    url = f"{BASE_URL}/property/{property_id}"
    response = requests.get(url, headers=HEADERS)
    try:
        return response.json()
    except ValueError as e:
        raise ValueError(f"Error parsing JSON response from property details: {e}")
    

In [12]:
def fetch_comparables(property_details, params):
    """Function to fetch HelloData comparables for a given property using a POST request."""
    url = f"{BASE_URL}/property/comparables"
    payload = {"subject": property_details}

    # Filter only the allowed parameters to be included in query string
    allowed_params = {
        "topN", "renovations", "maxDistance", "minDistance",
        "minNumberUnits", "maxNumberUnits", 
        "minYearBuilt", "maxYearBuilt", 
        "minNumberStories", "maxNumberStories"
    }

    # Construct filtered query parameters
    query_params = {k: v for k, v in params.items() if k in allowed_params}

    response = requests.post(url, headers=HEADERS, params=query_params, json=payload)

    if response.status_code == 200:
        return response.json()
    return None

In [13]:
def aggregate_comps(comps_json, property_details):
    # Extract subject property info
    subject_name = property_details.get('building_name')
    subject_address = f"{property_details.get('street_address', '')}, {property_details.get('city', '')}, {property_details.get('state', '')}"
    subject_year_built = property_details.get('year_built')
    subject_number_units = property_details.get('number_units')
    subject_number_stories = property_details.get('number_stories')

    # Initialize DataFrame with subject property
    comps_df = pd.DataFrame([{
        'Property Name': subject_name,
        'Address': subject_address,
        'Year Built': subject_year_built,
        'Number of Units': subject_number_units,
        'Number of Stories': subject_number_stories,
        'Reference': True
    }])

    # Append comps
    for i in range(len(comps_json)):
        cur = comps_json[i]

        property_name = cur.get('building_name')
        address = f"{cur.get('street_address', '')}, {cur.get('city', '')}, {cur.get('state', '')}"
        year_built = cur.get('year_built')
        number_units = cur.get('number_units')
        number_stories = cur.get('number_stories')

        comps_df = pd.concat([comps_df, pd.DataFrame([{
            'Property Name': property_name,
            'Address': address,
            'Year Built': year_built,
            'Number of Units': number_units,
            'Number of Stories': number_stories,
            'Reference': False
        }])], ignore_index=True)

    return comps_df


In [14]:
property_name = "Brooksedge"
lat = 39.924153
lon = -82.804660

In [15]:
property_data = fetch_property_data(property_name, lat = lat, lon = lon)
property_id = property_data[0]['id']

property_details = fetch_property_details(property_id)

comps = fetch_comparables(property_details, params=dict())['comparables']

In [17]:
display(comps)

[{'lat': 39.9024,
  'lon': -82.81536,
  'year_built': 2002,
  'number_units': 224,
  'number_stories': 3,
  'id': 'b4181053-5150-5ac9-a312-752f3ee9259f',
  'street_address': '3833 Abbie Lakes Drive',
  'street_address_alias': ['3833 Abbie Lakes Drive'],
  'city': 'Canal Winchester',
  'state': 'OH',
  'zip_code': '43110',
  'building_name': 'Abbie Lakes',
  'building_name_alias': ['Abbie Lakes', 'Abbie Lakes by Cortland'],
  'similarity_score': {'subtype': 1,
   'market': 0.5892405207950335,
   'distance': 0.8299019622307366,
   'building_amenities': 1,
   'unit_amenities': 0.8974358974358975,
   'number_units': 1,
   'number_stories': 0.8377514676971107,
   'vintage': 0.971012463126748,
   'quality': 0.936232171474526,
   'unit_mix': {'2': 0.9822158300571209},
   'overall': 0.9361707179016013,
   'distance_meters': 2651.470566538951,
   'distance_miles': 1.6475515220767216},
  'avg_quality': 0.6370755700922724,
  'distance_meters': 2651.470566538951,
  'distance_miles': 1.647551522076